In [1]:
import tkinter as tk
from tkinter import ttk, filedialog
import pandas as pd
import re
from bs4 import BeautifulSoup
import ollama
import threading
import queue
import json
import time

class ITTicketAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Analizador de Tickets IT")
        self.root.geometry("600x400")
        
        # Componentes GUI
        self.create_widgets()
        self.processing_queue = queue.Queue()
        self.llm_cache = {}
        
    def create_widgets(self):
        # Frame superior
        top_frame = ttk.LabelFrame(self.root, text="Entrada de datos")
        top_frame.pack(pady=10, padx=10, fill="x")
        
        self.file_path = tk.StringVar()
        ttk.Label(top_frame, text="Seleccionar archivo Excel:").pack(side="left")
        ttk.Entry(top_frame, textvariable=self.file_path, state="readonly").pack(side="left", expand=True, fill="x", padx=5)
        ttk.Button(top_frame, text="🖿", command=self.browse_file).pack(side="left")
        
        # Frame central
        center_frame = ttk.LabelFrame(self.root, text="Progreso")
        center_frame.pack(pady=10, padx=10, expand=True, fill="both")
        
        self.log_area = tk.Text(center_frame, wrap="none")
        scroll_y = ttk.Scrollbar(center_frame, orient="vertical", command=self.log_area.yview)
        scroll_x = ttk.Scrollbar(center_frame, orient="horizontal", command=self.log_area.xview)
        self.log_area.configure(yscrollcommand=scroll_y.set, xscrollcommand=scroll_x.set)
        
        self.log_area.grid(row=0, column=0, sticky="nsew")
        scroll_y.grid(row=0, column=1, sticky="ns")
        scroll_x.grid(row=1, column=0, sticky="ew")
        
        self.progress = ttk.Progressbar(center_frame, mode="determinate")
        self.progress.grid(row=2, column=0, columnspan=2, sticky="ew", pady=5)
        
        # Frame inferior
        bottom_frame = ttk.Frame(self.root)
        bottom_frame.pack(pady=10)
        ttk.Button(bottom_frame, text="▶ Ejecutar Proceso", command=self.start_processing).pack()

    def browse_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel Files", "*.xlsx")])
        if file_path:
            self.file_path.set(file_path)

    def start_processing(self):
        threading.Thread(target=self.process_tickets, daemon=True).start()

    def process_tickets(self):
        try:
            # Validación inicial
            file_path = self.file_path.get()
            if not file_path.endswith('.xlsx'):
                raise ValueError("Formato de archivo inválido")
            
            df = pd.read_excel(file_path)
            required_columns = ['Number', 'Priority', 'State', 'Short description', 'Task type']
            if not set(required_columns).issubset(df.columns):
                raise ValueError("Columnas requeridas faltantes")
            
            # Limpieza de datos
            df = self.clean_data(df)
            
            # Procesamiento con LLM
            total_tickets = len(df)
            self.progress["maximum"] = total_tickets
            
            for idx, row in df.iterrows():
                if idx % 5 == 0:
                    self.update_progress(idx)
                
                processed = self.process_ticket(row)
                df.loc[idx, ['DescripcionMistral', 'ResumenGemma3']] = processed
                
            self.save_results(df)
            
        except Exception as e:
            self.log(f"Error: {str(e)}")
            
    def clean_data(self, df):
        # Eliminar patrones con regex
        df['Description'] = df['Description'].apply(lambda x: re.sub(r'\[(ARGONAUTA


SyntaxError: unterminated string literal (detected at line 95) (3995271408.py, line 95)